<a href="https://colab.research.google.com/github/RajkumarBahalia/CountryList/blob/master/NiftyDailyMarketDepth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import files

# =============================
# 1. Check if file exists
# =============================
default_filename = "nifty_ohlc.csv"

if os.path.exists(default_filename):
    print(f"Found existing file: {default_filename}")
    filename = default_filename
else:
    print("File 'nifty_ohlc.csv' not found. Please upload your NIFTY OHLC CSV file:")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    print("Uploaded file:", filename)

# =============================
# 2. Robust CSV Read
# =============================
try:
    df = pd.read_csv(filename, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(filename, encoding="ISO-8859-1")

# =============================
# 3. Mandatory column check
# =============================
required_cols = ['Date', 'Close']
missing = [col for col in required_cols if col not in df.columns]

if missing:
    raise Exception(f"Missing required column(s): {missing}. Please check your CSV file.")

# =============================
# 4. Parse date column safely
# =============================

# Work with Date as string
date_raw = df['Date'].astype(str)

# First attempt: direct parse
parsed_dates = pd.to_datetime(date_raw, errors='coerce')

# If everything failed, try cleaning " GMT..." suffix (your file case)
if parsed_dates.isna().all():
    # Remove everything starting from " GMT"
    date_clean = date_raw.str.split(" GMT").str[0]
    parsed_dates = pd.to_datetime(date_clean, errors='coerce')

df['Date'] = parsed_dates

# Drop rows where Date could not be parsed
df = df.dropna(subset=['Date'])

# If after cleaning there are still no rows, warn
if df.empty:
    raise Exception("After parsing, 'Date' column is empty. Please check the date format in your CSV.")

# Sort by date
df = df.sort_values(by='Date')

# ============================================
# 3. Compute Point Change
# ============================================
prev_Close = df['Close'].shift(1)
change = df['Close'] - prev_Close
df['Nifty Chg %'] = ((df['Close'].pct_change()) * 100).round(2)

# ============================================
# 4. Compute Total Point Advance & Decline
# ============================================
df['Advance'] = change.apply(lambda x: round(x, 2) if x > 0 else 0)
df['Decline'] = change.apply(lambda x: round(x, 2) if x < 0 else 0)

# =============================
# 5. Calculate Daily % Change
# =============================
change = (df['Close'].pct_change() * 100).round(2)

# 4% Above Positive and 4% Below negative splits
df['Up_4%'] = change.apply(lambda x: x if x >= 4 else pd.NA)
df['Down_4%'] = change.apply(lambda x: x if x < 4 else pd.NA)

df['Up_4%'] = df['Up_4%'].round(2)
df['Down_4%'] = df['Down_4%'].round(2)

# === Compute Moving Averages ===
# === Store DMAs as VARIABLES (not in df) ===
dma10  = df['Close'].rolling(10).mean()
dma20  = df['Close'].rolling(20).mean()
dma50  = df['Close'].rolling(50).mean()
dma100 = df['Close'].rolling(100).mean()
dma200 = df['Close'].rolling(200).mean()

# === Calculate % Above / Below DMA ===
df['%_10DMA']  = ((df['Close'] - dma10)  / dma10)  * 100
df['%_20DMA']  = ((df['Close'] - dma20)  / dma20)  * 100
df['%_50DMA']  = ((df['Close'] - dma50)  / dma50)  * 100
df['%_100DMA'] = ((df['Close'] - dma100) / dma100) * 100
df['%_200DMA'] = ((df['Close'] - dma200) / dma200) * 100

# Round values
pct_cols = ['%_10DMA','%_20DMA','%_50DMA','%_100DMA','%_200DMA']
df[pct_cols] = df[pct_cols].round(2)

output_filename = "nifty_ohlc_Market_Depth_Data.csv"
df.to_csv(output_filename, index=False)

print("Saved:", output_filename)
